In [1]:
! git lfs install

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [2]:
! git clone https://github.com/timojl/clipseg 

Cloning into 'clipseg'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 215 (delta 62), reused 51 (delta 51), pack-reused 141
Receiving objects: 100% (215/215), 1.39 MiB | 26.33 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [3]:
! pip install diffusers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.0 MB/s eta 0:00:00


In [4]:
! pip install transformers -q -UU ftfy gradio  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.0 MB/s eta 0:00:

In [5]:
! pip install git+https://github.com/openai/CLIP.git -q

  Preparing metadata (setup.py) ... done


In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
%cd clipseg

/content/clipseg


In [8]:
! ls

datasets	     metrics.py		      score.py
environment.yml      models		      setup.py
evaluation_utils.py  overview.png	      supplementary.pdf
example_image.jpg    Quickstart.ipynb	      Tables.ipynb
experiments	     README.md		      training.py
general_utils.py     sample_rd64.png	      Visual_Feature_Engineering.ipynb
LICENSE		     sample_rd64_refined.png


In [9]:
import torch
import requests
import cv2 
from models.clipseg import CLIPDensePredT
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt

In [10]:
from io import BytesIO

from torch import autocast
import requests
import PIL
import torch
from diffusers import StableDiffusionInpaintPipeline as StableDiffusionInpaintPipeline

In [11]:
#! git lfs install

In [12]:
# load model
model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
model.eval();


100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 145MiB/s]


In [13]:
! wget https://owncloud.gwdg.de/index.php/s/ioHbRzFx6th32hn/download -O weights.zip
! unzip -d weights -j weights.zip

--2023-03-03 03:57:46--  https://owncloud.gwdg.de/index.php/s/ioHbRzFx6th32hn/download
Resolving owncloud.gwdg.de (owncloud.gwdg.de)... 134.76.23.45
Connecting to owncloud.gwdg.de (owncloud.gwdg.de)|134.76.23.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘weights.zip’

weights.zip             [    <=>             ]   9.68M  10.4MB/s    in 0.9s    

2023-03-03 03:57:47 (10.4 MB/s) - ‘weights.zip’ saved [10146905]

Archive:  weights.zip
 extracting: weights/rd16-uni.pth    
 extracting: weights/rd64-uni-refined.pth  
 extracting: weights/rd64-uni.pth    


In [14]:
# non-strict, because we only stored decoder weights (not CLIP weights)
model.load_state_dict(torch.load('/content/clipseg/weights/rd64-uni.pth', map_location=torch.device('cuda')), strict=False);


In [19]:

device = "cuda"
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/diffusers/pipelines/pipeline_utils.py:681: FutureWarning: You are using a legacy checkpoint for inpainting with Stable Diffusion, therefore we are loading the <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint_legacy.StableDiffusionInpaintPipelineLegacy'> class instead of <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline'>. For better inpainting results, we strongly suggest using Stable Diffusion's official inpainting checkpoint: https://huggingface.co/runwayml/stable-diffusion-inpainting instead or adapting your checkpoint CompVis/stable-diffusion-v1-4 to the format of https://huggingface.co/runwayml/stable-diffusion-inpainting. Note that we do not actively maintain the {StableDiffusionInpaintPipelineLegacy} class and will likely remove it in version 1.0.0.
  deprecate("StableDiffusionInpaintPipelineLegacy", "1.0.0", deprecation_message, standard_warn=False)
C

In [20]:
import gradio as gr

In [21]:
def prompt_paint(input_image, source_prompt, result_prompt):
  
  #input_image = Image.open(requests.get(image_url, stream=True).raw)

  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.Resize((512, 512)),
  ])
  
  img = transform(input_image).unsqueeze(0)

  prompts = [source_prompt]

  # predict
  with torch.no_grad():
      preds = model(img.repeat(len(prompts),1,1,1), prompts)[0]


  input_image.convert('RGB').resize((512, 512)).save("init_image.png", "PNG")
  
  
  filename = f"mask.png"
  
  plt.imsave(filename,torch.sigmoid(preds[0][0]))

  img2 = cv2.imread(filename)

  gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  (thresh, bw_image) = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)

  # For debugging only:
  cv2.imwrite(filename,bw_image)

  # fix color format
  cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)

  Image.fromarray(bw_image)

  init_image = Image.open('init_image.png')
  mask = Image.open('mask.png')

  with autocast("cuda"):
    images = pipe(prompt = result_prompt, init_image=init_image, mask_image=mask, strength=0.8)[0]

  return(images[0])


In [22]:
def prompt_paint_advanced(input_image, source_prompt, result_prompt, mask_precision, sd_strength):
  
  #input_image = Image.open(requests.get(image_url, stream=True).raw)

  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
      transforms.Resize((512, 512)),
  ])
  
  img = transform(input_image).unsqueeze(0)

  prompts = [source_prompt]

  # predict
  with torch.no_grad():
      preds = model(img.repeat(len(prompts),1,1,1), prompts)[0]


  input_image.convert('RGB').resize((512, 512)).save("init_image.png", "PNG")
  
  
  filename = f"mask.png"
  
  plt.imsave(filename,torch.sigmoid(preds[0][0]))

  img2 = cv2.imread(filename)

  gray_image = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  (thresh, bw_image) = cv2.threshold(gray_image, mask_precision, 255, cv2.THRESH_BINARY)

  # For debugging only:
  cv2.imwrite(filename,bw_image)

  cv2.cvtColor(bw_image, cv2.COLOR_BGR2RGB)

  Image.fromarray(bw_image)

  init_image = Image.open('init_image.png')
  mask = Image.open('mask.png')

  with autocast("cuda"):
    images = pipe(prompt = result_prompt, init_image=init_image, mask_image=mask, strength=sd_strength)[0]

  return(images[0])


In [23]:
with gr.Blocks() as demo:
    gr.Markdown("# Prompt Paint - Stable Diffusion")
    with gr.Tab("Prompt Paint - Basic"):
        with gr.Row():
            

            input_image = gr.Image(label = 'Upload your input image', type = 'pil')

            source_prompt = gr.Textbox(label="What is in the input image you want to change?")
            result_prompt = gr.Textbox(label="Replace it with?")

            image_output = gr.Image()
        
        image_button = gr.Button("Generate")

        image_button.click(prompt_paint, inputs=[input_image, source_prompt, result_prompt ], outputs=image_output)

    with gr.Tab("Prompt Paint - Advanced"):
        with gr.Row():
            

            input_image = gr.Image(label = 'Upload your input image', type = 'pil')

            source_prompt = gr.Textbox(label="What is in the input image you want to change?")
            result_prompt = gr.Textbox(label="Replace it with?")

                        
            mask_precision = gr.Slider(minimum = 50, maximum = 255, value = 70, label = 'Mask Precision')

            sd_strength = gr.Slider(minimum = 0, maximum = 1, value = 0.8, step = 0.1, label = 'Stable Diffusion Strength')
            

            image_output = gr.Image()
        
        image_button_advanced = gr.Button("Generate")

        image_button_advanced.click(prompt_paint_advanced, inputs=[input_image, source_prompt, result_prompt, mask_precision, sd_strength ], outputs=image_output)

demo.launch(debug = True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/usr/local/lib/python3.8/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint_legacy.py:607: FutureWarning: The `init_image` argument is deprecated and will be removed in version 0.14.0. Please use `image` instead of `init_image`.
  init_image = deprecate("init_image", "0.14.0", message, take_from=kwargs)


  0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint_legacy.py:607: FutureWarning: The `init_image` argument is deprecated and will be removed in version 0.14.0. Please use `image` instead of `init_image`.
  init_image = deprecate("init_image", "0.14.0", message, take_from=kwargs)


...

/usr/local/lib/python3.8/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint_legacy.py:607: FutureWarning: The `init_image` argument is deprecated and will be removed in version 0.14.0. Please use `image` instead of `init_image`.
  init_image = deprecate("init_image", "0.14.0", message, take_from=kwargs)


  0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint_legacy.py:607: FutureWarning: The `init_image` argument is deprecated and will be removed in version 0.14.0. Please use `image` instead of `init_image`.
  init_image = deprecate("init_image", "0.14.0", message, take_from=kwargs)


  0%|          | 0/40 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
